In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [7]:
#Set up the MLflow tracking server
mlflow.set_tracking_uri('http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/')
# Set or create an experiment
mlflow.set_experiment("Exp2: Handling Imbalanced Data")

<Experiment: artifact_location='s3://youtubecommentmlflow/228297973028603585', creation_time=1741879998278, experiment_id='228297973028603585', last_update_time=1741879998278, lifecycle_stage='active', name='Exp2: Handling Imbalanced Data', tags={}>

In [ ]:
df = pd.read_csv('C:/test/youtube_comment_sentiment_analysis/Artifacts/youtubecomment_preprocessing.csv')
df.shape

(15833, 2)

In [ ]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 1000  # Set max_features to 1000 for CountVectorizer

    # Step 1: Train-test split before vectorization and resampling
    X_train, X_test, y_train, y_test = train_test_split(df['Comment'], df['Sentiment'], test_size=0.2, random_state=42, stratify=df['Sentiment'])

    # Step 2: Vectorization using CountVectorizer, fit on training data only
    vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features) 
    X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
    X_test_vec = vectorizer.transform(X_test)  # Transform test data

    # Step 3: Handle class imbalance based on the selected method (only applied to the training set)
    if imbalance_method == 'class_weights':
        # Use class_weight in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = None  # Do not apply class_weight if using resampling

        # Resampling Techniques (only apply to the training set)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_CountVectorizer_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with CountVectorizer Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "CountVectorizer")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: CountVectorizer Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"C:/test/youtube_comment_sentiment_analysis/Artifacts/confusion_matrix_{imbalance_method}.csv"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()
        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_countvectorizer_trigrams_imbalance_{imbalance_method}")

# Step 6: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)


2025/03/14 16:50:55 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpi3ja7hpg\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 16:50:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_CountVectorizer_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585/runs/cb5735160af14d72b3805a7b14e879c3
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585


2025/03/14 16:51:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpbr9eaozj\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 16:51:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_CountVectorizer_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585/runs/deb94515741e4304956e24f63f76b05b
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585


2025/03/14 16:52:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmp5fzq_apt\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 16:52:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_CountVectorizer_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585/runs/87cdafcb1386416e99abf5e93d15c847
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585


2025/03/14 16:53:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmp0tmwvh6g\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 16:53:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_CountVectorizer_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585/runs/20f943a8808a4068848d7c678a6375ab
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585


2025/03/14 16:55:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpqtq1ixkq\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 16:55:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_CountVectorizer_Trigrams at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585/runs/f0f7fa7c08e24be19888c1241b67909e
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/228297973028603585
